# 환경설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/korquad

/content/drive/MyDrive/korquad


In [3]:
%pwd

'/content/drive/MyDrive/korquad'

In [4]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.3 MB 24 kB/s 
     |████████████████████████████████| 4.7 MB 33.3 MB/s 
     |████████████████████████████████| 6.6 MB 64.0 MB/s 
     |████████████████████████████████| 1.3 MB 57.9 MB/s 
     |████████████████████████████████| 120 kB 71.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.7.0+cu101 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.7.0+cu101 which is incompatible.


In [5]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.8 MB/s 
     |████████████████████████████████| 776.3 MB 12 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=5a2f780eda25dd51859c9ca7c1ca0948dbc671a2392875bf1bcc4edc1e59a460
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [6]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 32.7 MB/s 
     |████████████████████████████████| 453 kB 62.1 MB/s 


# scripts 생성

In [7]:
import json
from collections import OrderedDict
import re

file_data = OrderedDict()

file_data['version'] = 'KorQuAD_v1.0_dev'

title = '(title)'
context = '''
1 나는 혼자서 하기 힘든 일이 있어요.
2 OO은 혼자서 하기 힘든 일이 있을 때 불편함을 느껴요.
3 그럴 때 친구들은 선생님께 "저 도와주세요!"라고 말해요.
4 OO도 손을 들고, "저 도와주세요!"라고 말할 수 있어요!
5 선생님께 도와달라고 하는 것은 괜찮은 일이에요.
6 OO은 선생님이 도와주셔서 기분이 좋아져요!
7 선생님께서 도와주시고 나면, "감사합니다"라고 말해요.
8 선생님의 기분도 좋아져요.
'''

question = '.' # 삭제

context2 = context.replace("\n", "")
context2 = re.sub(r'[1-9]','', context2)

#### keybert

In [8]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [9]:
okt = Okt()

tokenized_doc = okt.pos(context2)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

In [10]:
n_gram_range = (1, 1)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

In [11]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([context2])
candidate_embeddings = model.encode(candidates)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [12]:
top_n = 3 # 상위 3개의 키워드를 출력
distances = cosine_similarity(doc_embedding, candidate_embeddings)
text = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(text)

['기분', '선생님', '친구']


#### json저장 -> 질문 출력

In [13]:
answer_start = [context2.find(text[0]), context2.find(text[1]), context2.find(text[2])]
# id가 질문마다 생성되는 거여서 어떻게 부여할지 생각 - 우선 0,1,2으로 해놓음

file_data['data'] = [{'paragraphs' : [{'qas' : [{'answers': [{'text': text[0], 'answer_start': answer_start[0]}], 'id': 0, 'question': question}, {'answers': [{'text': text[1], 'answer_start': answer_start[1]}], 'id': 1, 'question': question}, {'answers': [{'text': text[2], 'answer_start': answer_start[2]}], 'id': 2, 'question': question}], 'context' : context2}], 'title' : title}]

In [14]:
data = json.dumps(file_data, ensure_ascii=False)
data2 = json.loads(data)

print(data2)

{'version': 'KorQuAD_v1.0_dev', 'data': [{'paragraphs': [{'qas': [{'answers': [{'text': '기분', 'answer_start': 163}], 'id': 0, 'question': '.'}, {'answers': [{'text': '선생님', 'answer_start': 64}], 'id': 1, 'question': '.'}, {'answers': [{'text': '친구', 'answer_start': 59}], 'id': 2, 'question': '.'}], 'context': ' 나는 혼자서 하기 힘든 일이 있어요. OO은 혼자서 하기 힘든 일이 있을 때 불편함을 느껴요. 그럴 때 친구들은 선생님께 "저 도와주세요!"라고 말해요. OO도 손을 들고, "저 도와주세요!"라고 말할 수 있어요! 선생님께 도와달라고 하는 것은 괜찮은 일이에요. OO은 선생님이 도와주셔서 기분이 좋아져요! 선생님께서 도와주시고 나면, "감사합니다"라고 말해요. 선생님의 기분도 좋아져요.'}], 'title': '(title)'}]}


In [15]:
file_path = "data/text.json"

with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(data2, file)

In [16]:
MODEL_PATH = "artifacts/gpt2_2022.08.12_13.27.10/gpt2_step_75000.pth"
!python -m scripts.run_generate --model-path $MODEL_PATH --output-path decoded.txt

generate: 100% 3/3 [00:01<00:00,  1.75it/s]
